In [ ]:
import time
import logging
import os
import random
import pandas as pd
import numpy as np
import tensorflow as tf
import ConfigSpace as CS
import ConfigSpace.hyperparameters as CSH
import hpbandster.core.nameserver as hpns

In [ ]:
from hpbandster.optimizers import BOHB
from hpbandster.core.worker import Worker
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
tf.get_logger().setLevel('ERROR')

In [ ]:
df=pd.read_csv("ECTA 2023 Dataset.csv")
features = ['Relative Humidity', 'Wind Speed', 'Visibility', 'Pressure', 'Wind_Chill', 'Dewpoint Temp']
target = 'Temperature'

In [ ]:
df=df.dropna()
X=df[features].values
Y=df[target].values

In [ ]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2, random_state=40)
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [ ]:
def build_ann(lr,neurons1,neurons2):
    model=keras.Sequential([
        keras.layers.Input(shape=(X_train.shape[1],)),
        keras.layers.Dense(neurons1,activation='relu'),
        keras.layers.Dense(neurons2,activation='relu'),
        keras.layers.Dense(1,activation='linear')
    ])
    optimizer=keras.optimizers.Adam(learning_rate=lr)
    model.compile(optimizer=optimizer,loss='mean_squared_error',metrics=['mae'])
    return model

In [ ]:
class ANNWorker(Worker):
    def compute(self,config,budget,**kwargs):
        n1=int(config['neurons1'])
        n2=int(config['neurons2'])
        bs=int(config['batch_size'])
        lr=config['lr']

        model=build_ann(lr,n1,n2)
        model.fit(X_train,Y_train,epoch=int(budget),batch_size=bs,verbose=1)
        mae=model.evaluate(X_test,Y_test,verbose=1)
        return{
            'loss': mae,
            'info' : {'mae':mae,'loss':loss}
        }
    
    def get_configspace():
        cs = CS.ConfigurationSpace()
        cs.add_hyperparameter(CSH.UniformFloatHyperparameter('lr', 1e-4, 1e-2, log=True))
        cs.add_hyperparameter(CSH.UniformIntegerHyperparameter('neurons1', 32, 128))
        cs.add_hyperparameter(CSH.UniformIntegerHyperparameter('neurons2', 32, 128))
        cs.add_hyperparameter(CSH.UniformIntegerHyperparameter('batch_size', 16, 128))
        return cs


BOHB Setup and Run

In [ ]:
NS = hpns.NameServer(run_id='simple_bohb', host='localhost', port=None)
NS.start()

In [ ]:
worker = ANNWorker(run_id='simple_bohb')
worker.run(background=True)

bohb = BOHB(
    configspace=ANNWorker.get_configspace(),
    run_id='simple_bohb',
    nameserver='localhost',
    min_budget=5,
    max_budget=30
)
  # epochs from 5 to 30
res = bohb.run(n_iterations=10)

Print Best Result

In [ ]:
best_config = res.get_id2config_mapping()[res.get_incumbent_id()]['config']
print("Best Hyperparameters Found:")
print(best_config)

bohb.shutdown(shutdown_workers=True)
NS.shutdown()